In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('cleaned_book_store_data.csv', delimiter=',', encoding='ISO-8859-1')


In [3]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-M,User-ID,Age,Book-Rating
0,0002005018,Clara Callan,richard bruce wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,8,35,5
1,0002005018,Clara Callan,richard bruce wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,11676,35,8
2,0002005018,Clara Callan,richard bruce wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,67544,30,8
3,0002005018,Clara Callan,richard bruce wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,116866,35,9
4,0002005018,Clara Callan,richard bruce wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,123629,35,9


In [4]:
df.shape

(383852, 9)

# Rating Base Filtering

In [12]:
def recommend_top_books(df, top_n=5):
    # Group by ISBN to calculate the average rating and count of user ratings
    book_stats = df.groupby('ISBN').agg(
        avg_rating=('Book-Rating', 'mean'),
        user_count=('User-ID', 'count'),
        book_title=('Book-Title', 'first'),  # Get the book title (assumed all titles for ISBN are the same)
        book_author=('Book-Author', 'first')  # Get the book author
    ).reset_index()
    
    # Create a total score that combines rating and user count
    # Prioritize high ratings first, and break ties with user count
    book_stats['total_score'] = book_stats['avg_rating'] + book_stats['user_count']
    
    # Sort by average rating first (descending), and then by total score (descending)
    sorted_books = book_stats.sort_values(by=['avg_rating', 'total_score'], ascending=[False, False])
    
    # Get the top N books
    top_books = sorted_books.head(top_n)
    
    # Return relevant book details
    return top_books[['book_title', 'book_author', 'avg_rating', 'user_count']]

# Example usage
top_books_recommended = recommend_top_books(df, top_n=10)

# Display the top recommendations
top_books_recommended

,book_title,book_author,avg_rating,user_count
133078,Postmarked Yesteryear 30 Rare Holiday Postcards,pamela e. apkarian-russell,10.0,11
40968,The Sneetches and Other Stories,dr. seuss,10.0,8
111979,Natural California A Postcard Book,not applicable (na ),10.0,7
125303,Uncle John's Supremely Satisfying Bathroom Rea...,bathroom readers institute,10.0,7
41063,Oh the Thinks You Can Think I Can Read It All ...,dr. seuss,10.0,6
41417,The Lord of the Rings Leatherette Collector's ...,j. r. r. tolkien,10.0,6
41481,The Hobbit,j. r. r. tolkien,10.0,6
80339,Six by Seuss A Treasury of Dr Seuss Classics,seuss,10.0,6
1515,Hollywoodn't A Queer Postcard Book,laurence jaugey-paget,10.0,5
31547,The Ultimate Seduction Harlequin Blaze No 61,janelle denison,10.0,5


# Content Base Filtering

### suggest books to users based on the features and the user's preferences

#### Tf-IDF Vectorizer -> transform text data into numerical data
#### Tf -> how frequently a word appears in a doc
#### IDF -> meatures the importance of a word

In [19]:
# Group by 'Book-Author' and aggregate the 'Book-Title' into a list and count the number of books
authors_books_count = df.groupby('Book-Author')['Book-Title'].agg(lambda x: (list(x), len(x))).reset_index()

# Separate the list of books and the count into separate columns
authors_books_count[['Books', 'Count']] = pd.DataFrame(authors_books_count['Book-Title'].tolist(), index=authors_books_count.index)

# Filter authors with more than 5 books
filtered_authors = authors_books_count[authors_books_count['Count'] > 5][['Book-Author', 'Books']]
pd.set_option('display.max_colwidth', None)

# Rename columns for clarity
filtered_authors.columns = ['Author', 'Books']

# Display the result
filtered_authors

,Author,Books
14,a a milne,"[Piglet Meets a Heffalump, Piglet Meets a Heffalump, Piglet Meets a Heffalump, Christopher Robin Gives Pooh a Party, Kanga and Baby Roo Come to the Forest, Pooh Goes Visiting and Pooh and Piglet Nearl, Winnie the Pooh Storybook Treasury]"
27,a. a. attanasio,"[The Dragon and the Unicorn, The Dragon and the Unicorn, The Dragon and the Unicorn, The Dragon and the Unicorn, The Serpent and the Grail, The Last Legends of Earth, The Dragon and the Unicorn]"
31,a. a. milne,"[Pooh Giant Board Book, Winnie the Pooh, Winnie the Pooh, The House at Pooh Corner, The House at Pooh Corner, The House at Pooh Corner, The House at Pooh Corner, The House at Pooh Corner, The House at Pooh Corner, Walt Disneys Winnie the Pooh and Tigger Too Disney's Wonderful World of Reading 35, Walt Disneys Winnie the Pooh and Tigger Too Disney's Wonderful World of Reading 35, Walt Disneys Winnie the Pooh and Tigger Too Disney's Wonderful World of Reading 35, Walt Disneys Winnie the Pooh and Tigger Too Disney's Wonderful World of Reading 35, Walt Disneys Winnie the Pooh and Tigger Too Disney's Wonderful World of Reading 35, Pooh Goes Visiting A WinniethePooh Story Book, The Complete Tales Poems of WinnieThePooh, The Complete Tales Poems of WinnieThePooh, The Complete Tales Poems of WinnieThePooh, WinnieThePooh's Opposites, It's Tigger Time A PeekaPooh Book, It's Tigger Time A PeekaPooh Book, Where's Piglet Mouse Works PeekaPooh Book, Where's Piglet Mouse Works PeekaPooh Book, Where's Piglet Mouse Works PeekaPooh Book, Where's Piglet Mouse Works PeekaPooh Book, Where's Piglet Mouse Works PeekaPooh Book, Pooh Friendly Tales Friendly Tales Book, Pu Der Bar, Pu Der Bar, Pu Der Bar, Pu Der Bar, The Pooh Book of Quotations In Which Will Be Found Some Useful Information and Sustaining Thoughts by WinnieThePooh and His Friends, The Pooh Book of Quotations In Which Will Be Found Some Useful Information and Sustaining Thoughts by WinnieThePooh and His Friends, WinnieThePooh and Some Bees Winnie the Pooh Deluxe Picture Books, Winnie the Pooh and Tigger Too, Winnie the Pooh and Tigger Too, The Red House Mystery Dover Mystery Classics, The Red House Mystery Dover Mystery Classics, The Red House Mystery Dover Mystery Classics, The Red House Mystery Dover Mystery Classics, The Red House Mystery Dover Mystery Classics, The Complete Tales of WinnieThePooh, The Complete Tales of WinnieThePooh, WinnieThePooh, WinnieThePooh, WinnieThePooh, When We Were Very Young, When We Were Very Young, When We Were Very Young, Winnie the Pooh The Blustery Day Golden TellATale Book, Pooh's 101 Uses for a Honey Pot, Tigger Giant Board Book, House at Pooh Corner, The World of Pooh The Complete WinnieThePooh and the House at Pooh Corner, A Trick or Treat Surprise Costume PopUp Book, Now We Are Six, Now We Are Six, Now We Are Six, The World of Christopher Robin, WinnieThePooh Tells Time, The Pooh Story Book, Tiggers don't climb trees, Tiggers don't climb trees, Winnie Puh, WinnieThePooh and Some Bees Dutton Easy Reader, Hello Tigger Cloth and Board Book]"
38,a. b. yehoshua,"[Open Heart Harvest Book, A Journey to the End of the Millennium A Novel, A Journey to the End of the Millennium A Novel of the Middle Ages, The Liberated Bride, Mr Mani Harvest in Translation, Mr Mani Harvest in Translation]"
50,a. c. bhaktivedanta swami prabhupada,"[Journey of SelfDiscovery, Dharma The Way of Transcendence, Coming Back The Science of Reincarnation, Coming Back The Science of Reincarnation, The Science of SelfRealization, The Science of SelfRealization, Chant and Be Happy Based on Teachings of A C Bhaktivedanta Swami, BhagavadGita As It Is, Light of the Bhagavata, The Perfection of Yoga, Krsna the supreme personality of Godhead a summary study of rila Vyasadeva's rimadBhagavatam tenth canto, Bhagavad Gita As It Is Complete Edition, The Nectar of Instruction]"
...,...,...
60312,zoe valdes,"[Lobas De Mar, Lobas De Mar, Lobas De Mar, Lobas De Mar, Das tÃÃÂ¤gliche Nichts, Das tÃÃÂ¤glich

## Dividing into batches

In [26]:
# Define the batch size
batch_size = 10000

# Split the dataset into batches
batches = np.array_split(df, len(df) // batch_size)


C:\Users\94773\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import precision_score, recall_score, f1_score

In [30]:
# Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Process each batch
for i, batch in enumerate(batches):
    # Combine relevant text columns for TF-IDF
    combined_text = batch['Book-Title'] + ' ' + batch['Book-Author'] + ' ' + batch['Publisher']
    
    # Fit and transform the batch
    tfidf_matrix = tfidf.fit_transform(combined_text)
    
    # Save the TF-IDF matrix for later use
    batch['tfidf_matrix'] = list(tfidf_matrix)
    
    print(f'Processed batch {i+1}/{len(batches)}')


Processed batch 1/38
Processed batch 2/38
Processed batch 3/38
Processed batch 4/38
Processed batch 5/38
Processed batch 6/38
Processed batch 7/38
Processed batch 8/38
Processed batch 9/38
Processed batch 10/38
Processed batch 11/38
Processed batch 12/38
Processed batch 13/38
Processed batch 14/38
Processed batch 15/38
Processed batch 16/38
Processed batch 17/38
Processed batch 18/38
Processed batch 19/38
Processed batch 20/38
Processed batch 21/38
Processed batch 22/38
Processed batch 23/38
Processed batch 24/38
Processed batch 25/38
Processed batch 26/38
Processed batch 27/38
Processed batch 28/38
Processed batch 29/38
Processed batch 30/38
Processed batch 31/38
Processed batch 32/38
Processed batch 33/38
Processed batch 34/38
Processed batch 35/38
Processed batch 36/38
Processed batch 37/38
Processed batch 38/38


In [32]:
# Combine all batches into a single DataFrame
df_combined = pd.concat(batches)

# Create a mapping of book titles to indices
indices = pd.Series(df_combined.index, index=df_combined['Book-Title']).drop_duplicates()

# Example function to get recommendations by the same author
def get_recommendations_by_author(title, df, indices):
    idx = indices[title]
    author = df.loc[idx, 'Book-Author']
    author_books = df[df['Book-Author'] == author]
    return author_books[['Book-Title', 'Book-Author', 'Image-URL-M']]


In [34]:
# Example usage:
recommendations = get_recommendations_by_author('The Last Legends of Earth', df_combined, indices)
recommendations

,Book-Title,Book-Author,Image-URL-M
72920,The Dragon and the Unicorn,a. a. attanasio,http://images.amazon.com/images/P/0061057797.01.MZZZZZZZ.jpg
72921,The Dragon and the Unicorn,a. a. attanasio,http://images.amazon.com/images/P/0061057797.01.MZZZZZZZ.jpg
72922,The Dragon and the Unicorn,a. a. attanasio,http://images.amazon.com/images/P/0061057797.01.MZZZZZZZ.jpg
72923,The Dragon and the Unicorn,a. a. attanasio,http://images.amazon.com/images/P/0061057797.01.MZZZZZZZ.jpg
241759,The Serpent and the Grail,a. a. attanasio,http://images.amazon.com/images/P/0061059730.01.MZZZZZZZ.jpg
252631,The Last Legends of Earth,a. a. attanasio,http://images.amazon.com/images/P/0385263937.01.MZZZZZZZ.jpg
288512,The Dragon and the Unicorn,a. a. attanasio,http://images.amazon.com/images/P/0061092975.01.MZZZZZZZ.jpg


### Checking the accuracy

In [37]:
relevant_books = set(['The Serpent and the Grail', 'The Dragon and the Unicorn'])  
# Extract the recommended books' titles
recommended_books = set(recommendations['Book-Title'].values)

# Compute Precision, Recall, and F1-Score
true_positives = len(recommended_books.intersection(relevant_books))
false_positives = len(recommended_books.difference(relevant_books))
false_negatives = len(relevant_books.difference(recommended_books))

precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

Precision: 0.67
Recall: 1.00
F1-Score: 0.80


# Collaborative Filtering - Sayuni

## Recommend books to a user based on the preferences of other users who have rated similar books.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# Split the dataset into training and test sets (e.g., 80% training, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [15]:
def collaborative_recommendation(df, user_id, top_n=5, batch_size=5000, similarity_threshold=0.1):
    # Get unique user IDs and batch them
    user_ids = df['User-ID'].unique()
    num_batches = int(np.ceil(len(user_ids) / batch_size))
    user_id_batches = np.array_split(user_ids, num_batches)
    
    all_recommendations = []

    # Process each batch separately
    for batch_num, user_batch in enumerate(user_id_batches):
        print(f"Processing batch {batch_num + 1} of {num_batches}...")

        # Filter the dataframe to only include the users in this batch
        batch_train_df = df[df['User-ID'].isin(user_batch)]
        
        # Create the user-book matrix for the current batch
        user_book_matrix = batch_train_df.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating', aggfunc='mean').fillna(0)
        
        # Calculate user similarity using cosine similarity
        user_similarity = cosine_similarity(user_book_matrix)
        
        # Check if the target user is in the current batch
        if user_id in user_book_matrix.index:
            target_user_index = user_book_matrix.index.get_loc(user_id)
            
            # Get similarities for the target user
            user_similarities = user_similarity[target_user_index]
            similar_user_indices = user_similarities.argsort()[::-1][1:]

            recommended_books = []
            similarity_scores = []

            # Recommend books based on the top similar users
            for user_index in similar_user_indices:
                if user_similarities[user_index] >= similarity_threshold:  # Apply the threshold
                    rated_by_similar_user = user_book_matrix.iloc[user_index]
                    not_rated_by_target_user = (rated_by_similar_user != 0) & (user_book_matrix.iloc[target_user_index] == 0)

                    # Get the top recommended books from similar users
                    for book in user_book_matrix.columns[not_rated_by_target_user][:top_n]:
                        recommended_books.append(book)
                        similarity_scores.append(user_similarities[user_index])  # Store similarity score

            # Append unique recommendations to the final list
            unique_recommendations = list(set(recommended_books))
            all_recommendations.extend(zip(unique_recommendations, similarity_scores))

    # Create a DataFrame for recommended books and their similarity scores
    recommended_books_df = pd.DataFrame(all_recommendations, columns=['ISBN', 'Similarity Score'])
    
    # Get detailed information about the recommended books
    recommended_books_info = df[df['ISBN'].isin(recommended_books_df['ISBN'])][['ISBN', 'Book-Title', 'Book-Author', 'Image-URL-M']].drop_duplicates()
    
    # Merge details with similarity scores
    recommended_books_info = recommended_books_info.merge(recommended_books_df, on='ISBN', how='left')

    return recommended_books_info.head(top_n)

In [17]:
# Call the collaborative recommendation function
user_id = 67544  # user_id
top_n = 5        # Number of top recommendations to retrieve
batch_size = 5000  # Size of batches for processing

# Get the collaborative filtering recommendations for the user
collaborative_rec_output = collaborative_recommendation(df, user_id, top_n=top_n, batch_size=batch_size)

# Print the recommended books
print(f"Top {top_n} collaborative filtering recommendations for User {user_id}:")
collaborative_rec_output


Processing batch 1 of 14...
Processing batch 2 of 14...
Processing batch 3 of 14...
Processing batch 4 of 14...
Processing batch 5 of 14...
Processing batch 6 of 14...
Processing batch 7 of 14...
Processing batch 8 of 14...
Processing batch 9 of 14...
Processing batch 10 of 14...
Processing batch 11 of 14...
Processing batch 12 of 14...
Processing batch 13 of 14...
Processing batch 14 of 14...
Top 5 collaborative filtering recommendations for User 67544:


,ISBN,Book-Title,Book-Author,Image-URL-M,Similarity Score
0,074322678X,Where You'll Find Me And Other Stories,ann beattie,http://images.amazon.com/images/P/074322678X.0...,0.146334
1,0887841740,The Middle Stories,sheila heti,http://images.amazon.com/images/P/0887841740.0...,0.192778
2,1552041778,Jane Doe,r. j. kaiser,http://images.amazon.com/images/P/1552041778.0...,0.154526
3,1567407781,The Witchfinder Amos Walker Mystery Series,loren d. estleman,http://images.amazon.com/images/P/1567407781.0...,0.110327
4,1575663937,More Cunning Than Man A Social History of Rats...,robert hendrickson,http://images.amazon.com/images/P/1575663937.0...,0.240402


# Hybrid Filtering

In [109]:
def hybrid_recommendation(df, user_id, book_name, top_n=5, content_weight=0.5, collab_weight=0.5, batch_size=5000):
    # 1. Get content-based recommendations
    content_rec = get_recommendations_by_author(book_name, df, indices)
    
    # Print content recommendations
    print("Content Recommendations:")
    print(content_rec[['Book-Title', 'Book-Author', 'Image-URL-M']])
    
    # 2. Get collaborative filtering recommendations
    collab_rec = collaborative_recommendation(df, user_id, top_n=top_n, batch_size=batch_size)
    
    # Print collaborative recommendations
    print("\nCollaborative Recommendations:")
    print(collab_rec[['Book-Title', 'Book-Author', 'Image-URL-M', 'Similarity Score']])
    
    # 3. Combine scores
    combined_scores = {}
    
    # Add content-based scores
    for index, row in content_rec.iterrows():
        combined_scores[row['Book-Title']] = content_weight  # Assign the content weight
    
    # Add collaborative scores
    for index, row in collab_rec.iterrows():
        if row['Book-Title'] in combined_scores:
            combined_scores[row['Book-Title']] += collab_weight * row['Similarity Score']  # Add collaborative score
        else:
            combined_scores[row['Book-Title']] = collab_weight * row['Similarity Score']
    
    # Sort and get top recommendations
    sorted_combined_scores = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get recommended books
    top_recommendations = [item[0] for item in sorted_combined_scores[:top_n]]
    
    # Final recommendations DataFrame
    recommended_books = df[df['Book-Title'].isin(top_recommendations)][['Book-Title', 'Book-Author', 'Image-URL-M']]
    recommended_books['Combined Score'] = [combined_scores[title] for title in recommended_books['Book-Title']]
    
    return recommended_books

In [111]:
user_id = 67544
book_name = 'Open Heart Harvest Book'
hybrid_rec = hybrid_recommendation(df, user_id, book_name, top_n=5, content_weight=0.6, collab_weight=0.4, batch_size=5000)

hybrid_rec

Content Recommendations:
                                                                Book-Title  \
266031                                             Open Heart Harvest Book   
267896                      A Journey to the End of the Millennium A Novel   
289044  A Journey to the End of the Millennium  A Novel of the Middle Ages   
316041                                                 The Liberated Bride   
364268                                      Mr Mani Harvest in Translation   
364269                                      Mr Mani Harvest in Translation   

           Book-Author  \
266031  a. b. yehoshua   
267896  a. b. yehoshua   
289044  a. b. yehoshua   
316041  a. b. yehoshua   
364268  a. b. yehoshua   
364269  a. b. yehoshua   

                                                         Image-URL-M  
266031  http://images.amazon.com/images/P/0156004844.01.MZZZZZZZ.jpg  
267896  http://images.amazon.com/images/P/0385488823.01.MZZZZZZZ.jpg  
289044  http://images.amazon.com

,Book-Title,Book-Author,Image-URL-M,Combined Score
266031,Open Heart Harvest Book,a. b. yehoshua,http://images.amazon.com/images/P/0156004844.01.MZZZZZZZ.jpg,0.6
267896,A Journey to the End of the Millennium A Novel,a. b. yehoshua,http://images.amazon.com/images/P/0385488823.01.MZZZZZZZ.jpg,0.6
289044,A Journey to the End of the Millennium A Novel of the Middle Ages,a. b. yehoshua,http://images.amazon.com/images/P/0156011166.01.MZZZZZZZ.jpg,0.6
316041,The Liberated Bride,a. b. yehoshua,http://images.amazon.com/images/P/0151006539.01.MZZZZZZZ.jpg,0.6
364268,Mr Mani Harvest in Translation,a. b. yehoshua,http://images.amazon.com/images/P/0156627698.01.MZZZZZZZ.jpg,0.6
364269,Mr Mani Harvest in Translation,a. b. yehoshua,http://images.amazon.com/images/P/0156627698.01.MZZZZZZZ.jpg,0.6
